In [1]:
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import json
import requests
from newspaper import Article 
import socket
from urllib.error import URLError, HTTPError
import time
from urllib.parse import urlparse
import codecs

In [2]:
#Get an HTML file.
#Get news article from website.
import requests
url = 'https://query.nytimes.com/search/sitesearch/?action=click&contentCollection&region=TopBar&WT.nav=searchWidget&module=SearchSubmit&pgtype=Homepage#/electronic+health+records/from20070101to20170101/allresults/1/allauthors/newest/'
html_page = requests.get(url).text
print(html_page[:5000])

<!DOCTYPE html>
<html lang="en">
  <head>
    <title>NYTimes.com Search</title>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="robots" content="noarchive">
    <meta name="CG" content="Search">
    <meta name="SCG" content="cse">
    <meta name="PT" content="Search">
    <meta name="PST" content="Search Results">
    <meta name="PSST" content="">
    <meta name="PS" content="">
    <meta name="ttl" content="">
    <meta content="nyt-search" name="sourceApp" />                                   			
    <link rel="shortcut icon" href="https://static01.nyt.com/favicon.ico" />
    <link rel="stylesheet" type="text/css" href="https://s1.nyt.com/search/add/assets/16d75d84e70334b79c5187ff81f62a625eadde41/css/min.css">
    <!--[if IE]>
      <style type="text/css">
      @import url(https://graphics8.nytimes.com/css/0.1/screen/common/ie.css);
      </style>
    <![endif]-->
    <!--[if IE 6]>
      <style type="text/css">
      @import url(https://graph

In [3]:
 def get_nytURL(folder, num_page,search_q, begin_date, end_date):
        base = "http://api.nytimes.com/svc/search/v2/articlesearch"
        response_format = ".json"
        key = "b9378724fc554e96800c190b1fb5b45c"
      
        search_p = {"q":search_q, "api-key":key, "page":num_page, "begin_date":begin_date, "end_date":end_date}
        try:
            r = requests.get(base+response_format, params = search_p)
            #time.sleep(0.15)
            print(r.url)
            print(r.text[:500])
            #filename = "nyt-" + str(search_p) + ".json"
                
            file_path = folder + "nyt-" + str(search_p) + ".json"
            
            with open(file_path, "w") as jsonfile:
                jsonfile.write(r.text)
            
            r.raise_for_status()
            return r
        except requests.RequestException as e:
            print("[Error]: " + str(e))
            #print("failed to get from this topic: "+ str(search_q))
            return None
        except:
            print("[Error]: something else wrong in the try, not a RequestException.")
            return None


In [4]:
import os, errno

#query = "electronic health records"
query = "personal health records"
begin_date = "20070101"
end_date = "20170601"
#begin_date = "18520101"
#end_date = "1952101"

folder = "./" + query + "-" + str(begin_date) + "-" + str(end_date) + "-nyt-json/"
if not os.path.exists(folder):
    os.makedirs(folder)

#num_page = 0 means the first 10 results returned for the query
#num_page = 1 goes to the next 10 results
# num_page in range(0, 100) will return the top 100 results 
#for a givven query
for num_page in range(0,10):
    get_nytURL(folder, num_page, query, begin_date, end_date)
    time.sleep(5)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=personal+health+records&api-key=b9378724fc554e96800c190b1fb5b45c&page=0&begin_date=20070101&end_date=20170601
{"status":"OK","copyright":"Copyright (c) 2018 The New York Times Company. All Rights Reserved.","response":{"docs":[{"web_url":"https://www.nytimes.com/2017/05/29/arts/music/sonny-rollins-archives-schomburg-center.html","snippet":"This personal collection of hundreds of Mr. Rollins’s recordings and notes reveal his quest to improve his stamina and power as a saxophonist.","print_page":"1","blog":{},"source":"The New York Times","multimedia":[{"rank":0,"subtype":"thumbnail","caption":null,"credi
[Error]: something else wrong in the try, not a RequestException.
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=personal+health+records&api-key=b9378724fc554e96800c190b1fb5b45c&page=1&begin_date=20070101&end_date=20170601
{"status":"OK","copyright":"Copyright (c) 2018 The New York Times Company. All Rights Reserved.",

In [6]:
#Library to parse json objects
import json
import pandas
import glob

#query = "electronic health records"
query = "personal health records"
#begin_date = "20070101"
begin_date = "18520101"
end_date = "1952101"
#end_date = "20170601"
folder = "./" + query + "-" + str(begin_date) + "-" + str(end_date) + "-nyt-json/"

files = glob.glob(folder + "nyt-*.json")
df_nyt = pandas.DataFrame()
index = 0                  
for filename in files:
    print("file: " + filename)
    json_data = json.load(open(filename))
    articles = json_data['response']['docs']

    for article in articles:
        print(article)
        url = article["web_url"]
        snippet = article["snippet"]
        try:
            source = article["source"] 
        except: source = "None"
            
        headline = article["headline"]["main"]
        keywords = article["keywords"]
        keywords_list = []
        for keyword_obj in keywords:
            word = keyword_obj["name"] + ":" + keyword_obj["value"]
            keywords_list.append(word)
            
        date = article["pub_date"] 
        doctype = article["document_type"]
        try:
            new_desk = article["new_desk"]
        except: new_desk = "None"
            
        try:
            section_name = article["section_name"]
        except: section_name = "None"
            
        try:
            byline = article["byline"]
        except:
            byline = "None"
        
        doc_id = article["_id"]
        try:
            word_count = article["word_count"]
        except: word_count = "None"
        
        score = article["score"]
        
        print("url: " + url)
        print("snippet: " + snippet)
        print("source: " + source)
        print("headline: " + headline)
        print(keywords)
        print("date: " + date)
        print("doctype: " + doctype)
        print("new_desk: " + new_desk)
        print("section_name: " + section_name)
        print("id: " + doc_id)
        print("wc: " + str(word_count))
        print("score: " + str(score))
        
        df = pandas.DataFrame({"url": url, 
                                "snippet": snippet, 
                                "source": source, 
                                "headline": headline,
                                "keywords": ', '.join(keywords_list),
                                "doctype": doctype,
                                "new_desk": new_desk,
                                "section_name": section_name,
                                "id": doc_id,
                                "wc": word_count,
                                "score": score,
                                "date": date}, index=[0])
        
        #df.to_csv(query + "-" + str(begin_date) + "-" + str(end_date) + "-nyt-articles.csv", mode='a')
        df_nyt = df_nyt.append(df, ignore_index=True)
        index += 1          
        print(index)
    
print("with duplicates: " + str(df_nyt.shape))
#df_nyt = df_nyt.drop_duplicates()
print("no duplicates: " + str(df_nyt.shape))

#df_nyt.to_csv(query + "-" + str(begin_date) + "-" + str(end_date) + "-nyt-articles-no-duplicates.csv")


        

with duplicates: (0, 0)
no duplicates: (0, 0)
